In [ ]:
# Obtiene anomalías estandarizadas de viento y SST para el CLLJ.

import os

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr
import geopandas as gpd

import holoviews as hv
import geoviews as gv

import geoviews.feature as gf

from shapely.ops import clip_by_rect

# Se carga el motor gráfico de Geoviews.
gv.extension("matplotlib")
gv.output(size = 600)

In [ ]:
# Datos.

path_d = "../data/ERA5/"
path_c = "../results/onset/"
path_r = "../results/onset/graficas/"
path_map = "../data/Mapas/ne_50m_coastline/"
path_shp = "../data/Cuencas/Regiones_Hidrologicas_Administrativas/"
name_shp = "rha250kgw.shp"

vars = [ "u", "sst", "olr" ]
levels = [ "925" ]
region = [ "mexico" ]

# Se carga el contorno de México.
gdf = gpd.read_file(path_shp + name_shp)
# Se obtiene el contorno de los países.
gdf["boundary"] = gdf.boundary

# Media movil
d = 20

composite = [None] * 2

composite[0] = xr.open_dataset( path_c + "onset_"+  vars[1] + "_mean_" 
    + str(d) + "_dias.grib", engine = "cfgrib"
    ).drop( [ "step", "surface", "valid_time" ] )
composite[1] = xr.open_dataset( path_c + "onset_" + vars[0] + "_" 
    + levels[0] + "_mean_" + str(d) + "_dias.grib", engine = "cfgrib"
    ).drop( [ "step", "isobaricInhPa", "valid_time" ] )

zones = (-80, 12.5, -70, 17.5)

In [ ]:
# Índice de la SST en la zona del CLLJ.

fname_r = [ path_c + x + "_index.csv" for x in [ "CLLJ", "CLLJ_wind" ] ]

for i in range(2):

    # Seleccionamos la región.
    clip = composite[i].sel( longitude = slice( zones[0],
        zones[2] ), latitude = slice(zones[3], zones[1]) ).copy()

    # Promediamos y convertimos a información mensual.
    clip = clip.mean( ["latitude", "longitude" ]).to_dataframe()
    clip.index = clip.index.shift(periods = -18, freq = "H")
    df = clip.resample("1M", label = "left").mean()
    df.index += pd.to_timedelta(1, "D")

    # Anomalías estandarizadas.
    df = df.groupby(df.index.month).transform(
        lambda x: ( x - x.mean() ) / x.std() )

    # Reordenamos la tabla.
    df["Año"] = df.index.year
    df["Mes"] = df.index.month
    df.reset_index(inplace = True)
    df.drop("time", axis = 1, inplace = True)
    df = df.pivot( columns = "Mes", index = "Año"
        ).droplevel( level = 0, axis = 1 )

    df.to_csv(fname_r[i])

